<a href="https://colab.research.google.com/github/SumitJha406/Machine_learning_projects/blob/main/30_days_ML_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2/train.csv")

In [ ]:
df_train["kfold"] = -1

In [ ]:
df_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,-1
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,-1
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,-1
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,-1
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,-1


In [ ]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df_train)):
    df_train.loc[valid_indicies, "kfold"] = fold

In [ ]:
df_train

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,499993,B,B,A,A,B,D,A,E,A,I,0.697124,0.483452,0.297770,0.195821,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404,7.945605,4
299996,499996,A,B,A,C,B,B,A,E,E,F,0.446200,0.715135,0.610931,0.601730,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611,7.326118,3
299997,499997,B,B,A,C,B,C,A,E,G,F,0.544279,0.060937,0.590955,0.905308,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732,8.706755,1
299998,499998,A,B,A,C,B,B,A,E,E,I,0.300062,0.613118,0.285213,0.406851,0.805963,0.344404,0.424243,0.382028,0.468819,0.351036,0.288768,0.611169,0.380254,0.332030,7.229569,3


In [ ]:
df_train.loc[valid_indicies]

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
5,7,A,B,A,C,B,D,A,E,G,F,0.402056,0.837058,0.737991,0.778429,0.530250,0.392432,0.658169,0.997473,0.569874,0.960864,0.238050,0.316065,0.731729,0.694719,8.028558,4
13,18,B,A,A,C,B,D,A,E,A,M,0.814649,0.884882,0.725545,0.787947,0.276880,0.728022,0.553966,0.504254,0.555141,0.810336,0.896674,0.644984,0.639518,0.846561,7.428102,4
15,24,B,B,A,C,B,D,A,E,C,F,0.465176,0.207525,0.491816,0.616608,0.531611,0.617224,1.034829,0.409112,0.471035,0.574861,0.287672,0.713267,0.552684,0.162189,7.693611,4
18,27,A,B,A,C,B,B,A,B,E,F,0.162971,0.289554,0.600349,0.147401,0.286790,0.703116,0.569402,0.282601,0.447536,0.360268,0.705675,0.740588,0.782886,0.207666,8.017815,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299978,499959,A,A,A,C,B,D,A,E,G,H,1.025712,0.367695,0.224011,0.779099,0.529504,0.977901,0.542546,0.783931,0.835968,0.855563,0.867562,0.484421,0.641531,0.788356,8.342433,4
299981,499965,B,A,B,A,B,D,A,D,E,H,0.547278,0.635333,0.084690,0.389898,0.451891,0.614950,0.324285,0.699261,1.028858,0.750905,0.589536,0.638203,0.631909,0.793337,9.074018,4
299990,499980,B,A,A,C,B,D,A,E,A,F,0.536830,0.491664,0.360426,0.421627,0.590516,0.347792,0.809487,0.347889,0.608395,0.851322,0.880145,0.791714,0.854160,0.265987,8.115353,4
299993,499989,A,A,A,C,B,D,A,E,E,F,0.954027,0.494832,0.576918,0.420667,0.591178,1.009218,0.523628,0.654164,1.015928,0.592201,0.188211,0.164909,0.799694,0.439468,7.975959,4


In [ ]:
df_train.to_csv("/content/drive/MyDrive/Colab Notebooks/data2/train_folds.csv", index=False)

- ## In submission6 I have used max depth -2 ,lr = 0.12, 'subsample': 0.96,
    'colsample_bytree': 0.12,
    'max_depth': 3,
    'booster': 'gbtree', 
    'reg_lambda': 65.1,
    'reg_alpha': 15.9,
- ## In submission 7 I am gona use max depth -3 , lr = 0.12
- ## submission 8 was a average of submission 7 and submission file
- ## In submission 9 I am going to use max depth 3 and learning rate 0.1
- ## In submission 10 I am going to use colsample_bytree = 0.1

In [ ]:
from xgboost import XGBRegressor
xgb_params = {
    'n_estimators': 50000,
    'learning_rate': 0.1,
    'subsample': 0.96,
    'colsample_bytree': 0.1,
    'max_depth': 3,
    'booster': 'gbtree', 
    'reg_lambda': 65.1,
    'reg_alpha': 15.9,
    'random_state':40
    
}

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2/train_folds.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2/test.csv")
# sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in train_data.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
num_cols = [col for col in useful_features if "cont" in col]
df_test = test_data[useful_features]

In [ ]:
final_predictions = []
for fold in range(5):
    xtrain =  train_data[train_data.kfold != fold].reset_index(drop=True)
    xvalid = train_data[train_data.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    scaler = StandardScaler()
    xtrain[num_cols] = scaler.fit_transform(xtrain[num_cols])
    xvalid[num_cols] = scaler.transform(xvalid[num_cols])
    xtest[num_cols] = scaler.transform(xtest[num_cols])
    
    
    model = XGBRegressor(**xgb_params)
    model.fit(xtrain, ytrain,eval_set=[(xvalid, yvalid)],early_stopping_rounds= 35,verbose=True)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(f"fold: {fold}, rmse: {mean_squared_error(yvalid, preds_valid, squared=False)}")

Streaming output truncated to the last 5000 lines.
[894]	validation_0-rmse:0.720355
[895]	validation_0-rmse:0.720354
[896]	validation_0-rmse:0.720357
[897]	validation_0-rmse:0.720358
[898]	validation_0-rmse:0.720362
[899]	validation_0-rmse:0.720342
[900]	validation_0-rmse:0.720345
[901]	validation_0-rmse:0.720343
[902]	validation_0-rmse:0.720345
[903]	validation_0-rmse:0.720332
[904]	validation_0-rmse:0.720334
[905]	validation_0-rmse:0.720334
[906]	validation_0-rmse:0.720357
[907]	validation_0-rmse:0.720361
[908]	validation_0-rmse:0.720367
[909]	validation_0-rmse:0.720367
[910]	validation_0-rmse:0.720365
[911]	validation_0-rmse:0.720374
[912]	validation_0-rmse:0.720381
[913]	validation_0-rmse:0.720376
[914]	validation_0-rmse:0.720367
[915]	validation_0-rmse:0.720349
[916]	validation_0-rmse:0.720367
[917]	validation_0-rmse:0.720357
[918]	validation_0-rmse:0.720348
[919]	validation_0-rmse:0.720337
[920]	validation_0-rmse:0.720349
[921]	validation_0-rmse:0.720315
[922]	validation_0-rmse:0

KeyboardInterrupt: ignored

In [ ]:
final_predictions

[array([8.085243 , 8.3899765, 8.433418 , ..., 8.492945 , 8.201817 ,
        7.96995  ], dtype=float32),
 array([8.058225 , 8.4169445, 8.430688 , ..., 8.511635 , 8.166069 ,
        7.9600043], dtype=float32),
 array([8.0374975, 8.351259 , 8.445099 , ..., 8.50163  , 8.140912 ,
        8.004791 ], dtype=float32),
 array([8.050475 , 8.390795 , 8.448244 , ..., 8.504422 , 8.129131 ,
        7.9847074], dtype=float32),
 array([8.060959, 8.375076, 8.377632, ..., 8.526815, 8.167557, 7.940564],
       dtype=float32)]

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2/sample_submission.csv")

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.target = preds
sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/data2/submission10.csv", index=False)


### submission 8 using average of submission.csv and submission7.csv
- submission.csv is taken from kaggle users 0|1


In [ ]:
t1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data2/submission7.csv')
t2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dmata2/submission.csv')

In [ ]:
t1,t2 = t1.drop('id',axis=1),t2.drop('id',axis=1)

In [ ]:

t2 = t2.to_numpy()

t = np.stack((t1,t2),axis = 1)

array([[8.068678 ],
       [8.3747225],
       [8.438189 ],
       ...,
       [8.498619 ],
       [8.170473 ],
       [7.980207 ]])

In [ ]:
tmean = t.mean(axis =1)
tmean

In [ ]:
sample_submission.target = tmean

sample_submission

In [ ]:
sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/data2/submission8.csv", index=False)

In [ ]:
final_predictions = []
for fold in range(5):
    xtrain =  train_data[train_data.kfold != fold].reset_index(drop=True)
    xvalid = train_data[train_data.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    # ordinal_encoder = OrdinalEncoder()
    # xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    # xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    # xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown="ignore")
    xtrain_ohe = ohe.fit_transform(xtrain[object_cols])
    xvalid_ohe = ohe.transform(xvalid[object_cols])
    xtest_ohe = ohe.transform(xtest[object_cols])
    
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns=[f"ohe_{i}" for i in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns=[f"ohe_{i}" for i in range(xvalid_ohe.shape[1])])
    xtest_ohe = pd.DataFrame(xtest_ohe, columns=[f"ohe_{i}" for i in range(xtest_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis=1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis=1)
    xtest = pd.concat([xtest, xtest_ohe], axis=1)
    
    # this part is missing in the video:
    xtrain = xtrain.drop(object_cols, axis=1)
    xvalid = xvalid.drop(object_cols, axis=1)
    xtest = xtest.drop(object_cols, axis=1)
    # missing part ends
    
    
    # scaler = StandardScaler()
    # xtrain[num_cols] = scaler.fit_transform(xtrain[num_cols])
    # xvalid[num_cols] = scaler.transform(xvalid[num_cols])
    # xtest[num_cols] = scaler.transform(xtest[num_cols])
    
    
    model = XGBRegressor(**xgb_params)
    model.fit(xtrain, ytrain,eval_set=[(xvalid, yvalid)],early_stopping_rounds= 50,verbose=True)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(f"fold: {fold}, rmse: {mean_squared_error(yvalid, preds_valid, squared=False)}")

Streaming output truncated to the last 5000 lines.
[1567]	validation_0-rmse:0.718197
[1568]	validation_0-rmse:0.718192
[1569]	validation_0-rmse:0.718192
[1570]	validation_0-rmse:0.718192
[1571]	validation_0-rmse:0.718193
[1572]	validation_0-rmse:0.718193
[1573]	validation_0-rmse:0.718195
[1574]	validation_0-rmse:0.718197
[1575]	validation_0-rmse:0.718195
[1576]	validation_0-rmse:0.718196
[1577]	validation_0-rmse:0.718194
[1578]	validation_0-rmse:0.718195
[1579]	validation_0-rmse:0.718194
[1580]	validation_0-rmse:0.718193
[1581]	validation_0-rmse:0.718187
[1582]	validation_0-rmse:0.718182
[1583]	validation_0-rmse:0.718181
[1584]	validation_0-rmse:0.718182
[1585]	validation_0-rmse:0.718182
[1586]	validation_0-rmse:0.718181
[1587]	validation_0-rmse:0.718179
[1588]	validation_0-rmse:0.718182
[1589]	validation_0-rmse:0.718181
[1590]	validation_0-rmse:0.718182
[1591]	validation_0-rmse:0.718179
[1592]	validation_0-rmse:0.718181
[1593]	validation_0-rmse:0.718181
[1594]	validation_0-rmse:0.7181

In [ ]:
final_predictions

[array([8.091248 , 8.399517 , 8.403047 , ..., 8.46525  , 8.2282715,
        7.981871 ], dtype=float32),
 array([8.04914  , 8.386332 , 8.41939  , ..., 8.463569 , 8.196163 ,
        7.9598675], dtype=float32),
 array([8.028653 , 8.3355255, 8.413646 , ..., 8.492315 , 8.128363 ,
        7.962638 ], dtype=float32),
 array([8.014851 , 8.403837 , 8.4411335, ..., 8.51517  , 8.1393175,
        8.0020275], dtype=float32),
 array([8.046828, 8.368953, 8.41013 , ..., 8.546832, 8.202816, 7.947265],
       dtype=float32)]

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2/sample_submission.csv")

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.target = preds
sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/data2/submission12.csv", index=False)
